imports

In [2]:
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd
import datetime

cookies (probably unimportant)

In [3]:
def load_cookies(file_path):
  # Load in cookies
  cookies_raw = json.load(open(file_path, 'r'))
  return {cookie['name']: cookie['value'] for cookie in cookies_raw}

def fetch_data(session, url, cookies):
  return session.get(url, cookies=cookies)

parsing

In [6]:
def parse_data(soup, url):
  question_count = 28

  responses = [[] for _ in range(question_count)]

  # Extract response counts for each question
  for i in range(question_count):
      question_id = f"ContentPlaceHolder1_dlQuestionnaire_rptChoices_{i+1}_rbSelect_"
      question_responses = soup.find_all(id=lambda x: x and x.startswith(question_id))
      for response in question_responses:
        responses[i].append(response.contents[0])
  
  data = {"Instructor": soup.find(id="ContentPlaceHolder1_lblInstructorName").text.replace('"','').replace(",",""),
         "Course": soup.find(id="ContentPlaceHolder1_lblCourseDescription").text}
  
  # grades received
  table = soup.find('table', id="ContentPlaceHolder1_tblGradesReceived")
  grades = []

  if table:
    for td in table.find('tbody').find('tr').find_all('td'):
      grades.append(td.text)

  data["Grades"] = "/".join(grades)
  
  # Convert the list of responses into a dictionary with question numbers as keys
  end = {f'Question {i+1}': "/".join(responses[i]) for i in range(question_count)}

  # Add link at end
  end['Link'] = url

  data.update(end)

  return data

def fetch_and_parse_data(session, url, cookies, headers):
  response = session.get(url, cookies=cookies, headers=headers)
  if response.status_code == 200:
      soup = BeautifulSoup(response.text, 'html.parser')
      return parse_data(soup, url)
  else:
      print(f"Error fetching {url}: Status code {response.status_code}")
    

In [5]:
def append_to_csv(data, file_path):
    pd.DataFrame([data]).to_csv(file_path, mode='a', index=False, header=False)

In [7]:
def main():
  max_attempts = 20  # Set a maximum number of attempts to avoid infinite loops
  attempt = 0

  while attempt < max_attempts:
    try:
      cookies = load_cookies('cape.ucsd.edu_cookies.json')
      headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36',
        'Accept-Encoding': '*',
        'Connection': 'keep-alive'
      }

      csv_file_path = 'master.csv'

      cur = pd.read_csv(csv_file_path).shape[0]
      df = pd.read_csv('capes_data.csv')

      session = requests.Session()
      session.headers.update(headers)
      print(f"Attempt {attempt+1}: " + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f"))

      for index in range(cur, 30000):  # Replace 'Link' with the actual column name containing the links
        url = df['Evalulation URL'][index]
        # only print every 10
        if ((index+1) % 10 == 0):
          print(f"{index+1}: " + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f"))

        response = fetch_data(session, url, cookies)
        soup = BeautifulSoup(response.text, 'html.parser')
        append_to_csv(parse_data(soup, url), csv_file_path)
      # If this block executes successfully, break out of the loop
      attempt = max_attempts  # Ensure the loop exits after successful execution
    except Exception as e:
        # Handle specific exceptions you're expecting and decide to retry
        print(f"Error encountered: {e}. Attempting to restart...")
        attempt += 1
        if attempt == max_attempts:
            print("Maximum attempts reached. Exiting.")